In [2]:
import os
import re
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from googletrans import Translator

### **TRANSLATE CSV**

In [19]:
def translate_dataset(file_path, output_path, text_column, src_lang='en', dest_lang='vi'):
    translator = Translator()
    data = pd.read_csv(file_path)
    def translate_to_vietnamese(text):
        try:
            translated = translator.translate(text, src=src_lang, dest=dest_lang)
            return translated.text
        except Exception as e:
            return f"Dịch lỗi: {str(e)}"

    # Apply translation to the specified column
    if text_column in data.columns:
        data[text_column] = data[text_column].apply(translate_to_vietnamese)
    else:
        raise ValueError(f"Column '{text_column}' not found in the dataset.")

    # Save the translated dataset
    data.to_csv(output_path, index=False)
    print(f"Translated dataset saved to: {output_path}")


In [3]:
df = pd.read_csv('translated_emails_vietnamese.csv')
df.columns = ['email', 'label']
df.head(10)

,email,label
0,Chủ đề: Tự nhiên không thể cưỡng lại được bản ...,1
1,Chủ đề: Súng giao dịch chứng khoán Fanny là Me...,1
2,Chủ đề: Những ngôi nhà mới không thể tin được ...,1
3,Chủ đề: 4 in màu Yêu cầu đặc biệt Thông tin bổ...,1
4,"Chủ đề: Không có tiền, lấy phần mềm đĩa CD từ ...",1
5,"Chủ đề: Nnews tuyệt vời Xin chào, Chào mừng bạ...",1
6,"Dịch lỗi: the JSON object must be str, bytes o...",1
7,Chủ đề: Tiết kiệm tiền của bạn mua được thứ nà...,1
8,Chủ đề: Không được gửi: Kinh doanh tại nhà dàn...,1
9,Chủ đề: Tiết kiệm tiền của bạn mua được thứ nà...,1


In [9]:
df_filtered = df[~df['email'].str.startswith('Dịch lỗi')]
print(df_filtered)
df_filtered.to_csv('filtered_dataset.csv', index=False)

                                                  email  label
0     Chủ đề: Tự nhiên không thể cưỡng lại được bản ...      1
1     Chủ đề: Súng giao dịch chứng khoán Fanny là Me...      1
2     Chủ đề: Những ngôi nhà mới không thể tin được ...      1
3     Chủ đề: 4 in màu Yêu cầu đặc biệt Thông tin bổ...      1
4     Chủ đề: Không có tiền, lấy phần mềm đĩa CD từ ...      1
...                                                 ...    ...
5723  Chủ đề: Re: Phí nghiên cứu và phát triển cho G...      0
5724  Chủ đề: Re: Biên lai từ Visit Jim, một lần nữa...      0
5725  Chủ đề: Re: Cập nhật nghiên cứu trường hợp Enr...      0
5726  Chủ đề: Re: lãi David, làm ơn, hãy gọi Shirley...      0
5727  Chủ đề: Tin tức: Aurora 5.2 Cập nhật phiên bản...      0

[5477 rows x 2 columns]


In [10]:
df_filtered.shape

(5477, 2)

In [11]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_filtered['email'])

term_frequencies = X.sum(axis=0).A1
vocab = vectorizer.get_feature_names_out()

freq_dict = dict(zip(vocab, term_frequencies))

sorted_terms = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)[:3000]

top_words = [word for word, freq in sorted_terms]

output_txt_path = 'vietnamese_words.txt'
with open(output_txt_path, 'w', encoding='utf-8') as f:
    for word in top_words:
        f.write(word + '\n')

print(f"Top 3000 terms saved to: {output_txt_path}")


Top 3000 terms saved to: vietnamese_words.txt


In [13]:
def count_words(df, words_file='vietnamese_words.txt'):
    with open(words_file, 'r') as file:
        words = [line.strip() for line in file.readlines()]
    word_counts = pd.DataFrame()
    for word in words:
        word_counts[word] = df['email'].apply(lambda x: x.lower().split().count(word))
    word_counts['label'] = df['label']
    word_counts.to_csv('dt1_split_vietnamese.csv', index=False)

In [14]:
count_words(df_filtered)

C:\Users\Phu Nguyen\AppData\Local\Temp\ipykernel_7700\3399097761.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  word_counts[word] = df['email'].apply(lambda x: x.lower().split().count(word))
C:\Users\Phu Nguyen\AppData\Local\Temp\ipykernel_7700\3399097761.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  word_counts[word] = df['email'].apply(lambda x: x.lower().split().count(word))
C:\Users\Phu Nguyen\AppData\Local\Temp\ipykernel_7700\3399097761.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

In [4]:
df = pd.read_csv('dt1_split_vietnamese.csv')
df.head(10)

,tôi,của,bạn,và,có,cho,một,các,được,chúng,...,adam,advantaqes,aga,alamo,at,bangalore,bbo,blachman,bldg,label
0,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,2,0,1,0,0,0,1,0,3,...,0,0,0,0,0,0,0,0,0,0
3,0,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,2,1,1,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5,0,2,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,2,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,0,1,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
9,0,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
X = df.drop(columns=['label'])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = [
    ('Naive Bayes', MultinomialNB()),
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('SVM', SVC(kernel='linear')),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42))
]

model_dir = 'MODEL'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

results = []

for model_name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    results.append({
        'Model': model_name,
        'Accuracy': acc,
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score']
    })
    joblib.dump(model, os.path.join(model_dir, f'{model_name.lower().replace(" ", "_")}_model2.joblib'))

results_df = pd.DataFrame(results)
results_df

,Model,Accuracy,Precision,Recall,F1-Score
0,Naive Bayes,0.970803,0.972961,0.970803,0.971214
1,Logistic Regression,0.975365,0.975470,0.975365,0.975408
2,SVM,0.970803,0.970803,0.970803,0.970803
3,Random Forest,0.952555,0.953849,0.952555,0.951226
4,Gradient Boosting,0.955292,0.955956,0.955292,0.954271


### ***test_data2***

In [20]:
file_path = r'DATA/emails_test2.csv'
output_path = 'translated_emails_vietnamese2.csv'

In [23]:
translate_dataset(file_path, output_path, 'email')

Translated dataset saved to: translated_emails_vietnamese2.csv


In [24]:
df = pd.read_csv('translated_emails_vietnamese2.csv')
df['label'] = df['label'].apply(lambda x: 1 if x == 'spam' else 0)
df.head()

,email,label
0,Nâng cấp lên kế hoạch cao cấp của chúng tôi để...,0
1,Chúc mừng ngày lễ từ đội của chúng tôi!Chúc bạ...,0
2,Chúng tôi đang tuyển dụng!Kiểm tra các cơ hội ...,0
3,Tài khoản Amazon của bạn đã bị khóa.Nhấn vào đ...,1
4,Ý kiến ​​của bạn quan trọng!Lấy khảo sát của c...,0


In [25]:
count_words(df)

C:\Users\Phu Nguyen\AppData\Local\Temp\ipykernel_7700\3399097761.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  word_counts[word] = df['email'].apply(lambda x: x.lower().split().count(word))
C:\Users\Phu Nguyen\AppData\Local\Temp\ipykernel_7700\3399097761.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  word_counts[word] = df['email'].apply(lambda x: x.lower().split().count(word))
C:\Users\Phu Nguyen\AppData\Local\Temp\ipykernel_7700\3399097761.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

In [26]:
df = pd.read_csv('dt1_split_vietnamese.csv')
df.head()

,tôi,của,bạn,và,có,cho,một,các,được,chúng,...,adam,advantaqes,aga,alamo,at,bangalore,bbo,blachman,bldg,label
0,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,2,0,1,0,0,0,1,0,3,...,0,0,0,0,0,0,0,0,0,0
3,0,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,2,1,1,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [27]:
model = joblib.load(r'MODEL/naive_bayes_model2.joblib')

In [28]:
X = df.drop(columns=['label'])
y = df['label']  
y_pred = model.predict(X)

print(f"Độ chính xác của:", accuracy_score(y, y_pred))
print(f"Báo cáo phân loại của:\n", classification_report(y, y_pred))

Độ chính xác của: 0.6703910614525139
Báo cáo phân loại của:
               precision    recall  f1-score   support

           0       0.94      0.44      0.60       100
           1       0.58      0.96      0.72        79

    accuracy                           0.67       179
   macro avg       0.76      0.70      0.66       179
weighted avg       0.78      0.67      0.65       179

